# Data Base

Florian EPAIN
20105403, MASTER IL group 3

## Belatedly Installation

We can visualize blank `.csv` file using `vscode` and the extension `Rainbow CSV`.

### Docs

- I used the [Psycopg2 tutorials](https://www.psycopg.org/docs/install.html#quick-install) to connect my notebook to my database.
- [SQL Datatypes](https://www.w3schools.com/sql/sql_datatypes.asp)
- [Install Custom postgres](https://ravinderfzk.medium.com/install-postgresql-and-pgadmin4-in-arch-linux-eb013b45540f)

In [ ]:
%pip install psycopg2-binary

In [ ]:
%pip install -U pip

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
%pip install duckdb==0.9.1

⡿ installing duckdb from spec 'duckdb==0.9.1'
No apps associated with package duckdb or its dependencies. If you are
attempting to install a library, pipx should not be used. Consider using pip
or a similar tool instead.


Archlinux build:
- `pacman -S python-psycopg2`
- `pacman -S python-pipx`
- `pacman -S python-duckdb`

In [31]:
# If the Server postgres is down
import duckdb
cursor_duckdb = duckdb.connect()
# print(cursor_duckdb.execute('SELECT 42').fetchall())

In [1]:
import psycopg2
from getpass import getpass
  
# Connection establishment
# conn = psycopg2.connect( 
#     database="fepain", 
#     user='fepain', 
#     password=getpass(), 
#     host='chausey.istic.univ-rennes1.fr', 
#     port='5432'
# )


# localhost Connection
conn = psycopg2.connect( 
    database='m1-db-lab', 
    user='postgres', 
    password=getpass(), 
    host='localhost', 
    port='5432'
)
  
conn.autocommit = True
  
# Creating a cursor object 
cursor_postgres = conn.cursor()

In [16]:
# Query to create a table 
sql = ''' CREATE TABLE TripData 
( 
    medallion VARCHAR(32), 
    hack_licence VARCHAR(32), 
    vendor_id VARCHAR(10), 
    rate_code INT,
    store_and_fwd_flag VARCHAR(10),
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count INT,
    trip_time_in_secs INT,
    trip_distance FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT
); '''
  
# --- Executing above query ---
cursor_postgres.execute(sql) # postgres
cursor_duckdb.execute(sql) # duckdb
print("Table has been created successfully D:")

Table has been created successfully D:


In [30]:
# WARNING: Display the WHOLE table 

cursor_postgres.execute('SELECT * FROM TripData')
print("PostgreSQL:")
print(cursor_postgres.fetchall())

cursor_duckdb.execute('SELECT * FROM TripData')
print("DuckDB:")
print(cursor_duckdb.fetchall())

PostgreSQL:
[]
DuckDB:
[]


In [ ]:
# First Try
sql = ''' COPY TripDataSample FROM 'trip_date-sample.csv' With CSV HEADER DELIMITER as ','; '''
# cursor.execute(sql)
# print(cursor.fetchall())

---

We now import the `csv` file, using the [Cursor Copy - tutorials](https://www.psycopg.org/docs/cursor.html#cursor.copy_from) and the [COPY docs](https://www.postgresql.org/docs/current/sql-copy.html). I can't use `copy_from()`, I will use `copy_expert()` as mentionned in this [github issue](https://github.com/psycopg/psycopg2/issues/1294) in POSTGRESQL

---

In [42]:
# Import Sample CSV in PostgreSQL
csv_file_path = "assets/data/trip_data-sample.csv"
# cursor_postgres.copy_from(f, 'TripData')

with open(csv_file_path, 'r') as f:
    cursor_postgres.copy_expert(
    "COPY TripData FROM STDIN With CSV HEADER DELIMITER as ','",
    f)

# Display the table 

# cursor_postgres.execute('SELECT * FROM TripData')
# print(cursor_postgres.fetchall())

[('89D227B655E5C82AECF13C3F540D4CF4', 'BA96DE419E711691B9445D6A6307C170', 'CMT', 1, 'N', datetime.datetime(2013, 1, 1, 15, 11, 48), datetime.datetime(2013, 1, 1, 15, 18, 10), 4, 382, 1.0, -73.978165, 40.757977, -73.989838, 40.751171), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 6, 0, 18, 35), datetime.datetime(2013, 1, 6, 0, 22, 54), 1, 259, 1.5, -74.006683, 40.731781, -73.994499, 40.75066), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 5, 18, 49, 41), datetime.datetime(2013, 1, 5, 18, 54, 23), 1, 282, 1.1, -74.004707, 40.73777, -74.009834, 40.726002), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786310', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 23, 54, 15), datetime.datetime(2013, 1, 7, 23, 58, 20), 2, 244, 0.7, -73.974602, 40.759945, -73.984734, 40.759388), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786

In [ ]:
# Import Sample CSV in DuckDB

# cursor_duckdb.execute('SELECT * FROM \'trip_data-sample.csv\';')
# print(cursor_duckdb.fetchall())

sql = ''' COPY TripData FROM 'assets/data/trip_data-sample.csv' With CSV HEADER DELIMITER as ','; '''
cursor_duckdb.execute(sql)
print(cursor_duckdb.fetchall())

In [ ]:
# Drop the table

cursor_postgres.execute('DROP TABLE TripData')
cursor_duckdb.execute('DROP TABLE TripData')

---

Big CSV Import

---

In [18]:
# Import Big CSV in PostgreSQL
csv_file_path = "assets/data/trip_data/trip_data_1.csv"
# cursor.copy_from(f, 'TripData')

with open(csv_file_path, 'r') as f:
    cursor_postgres.copy_expert(
    "COPY TripData FROM STDIN With CSV HEADER DELIMITER as ','",
    f)

medallion,hack_licence,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude

In [ ]:
from sys import stdout

# Display attributes
cursor_postgres.copy_expert("COPY TripData TO STDOUT WITH CSV HEADER", stdout)

medallion,hack_licence,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude


In [34]:
# Import Big CSV in DuckDB

# cursor_duckdb.execute('SELECT * FROM \'trip_data_1.csv\';')
# print(cursor_duckdb.fetchall())

sql = ''' COPY TripData FROM 'assets/data/trip_data/trip_data_1.csv' With CSV HEADER DELIMITER as ','; '''
cursor_duckdb.execute(sql)
print(cursor_duckdb.fetchall())

[(14776615,)]


## Analyze Queries


- [`EXPLAIN` docs](https://www.postgresql.org/docs/current/sql-explain.html)
- <https://explain.dalibo.com/>
- Use [`CLUSTER`](https://www.postgresql.org/docs/current/sql-cluster.html)
- Use [`CREATE INDEX name on Table (column_to_index)`](https://www.postgresql.org/docs/current/sql-createindex.html)
- use `EXPLAIN VERBOSE` / `BUFFER` to exploit `EXPLAIN`
  - add `ANALYZE` to display only graph
- optimize request with duckDB

### Optimize PostgreSQL

In [5]:
optimize = 'CREATE INDEX index_medallion on TripData (medallion);'
cursor_postgres.execute(optimize)

In [8]:
optimize = 'CREATE INDEX index_passenger on TripData (passenger_count);'
cursor_postgres.execute(optimize)

In [ ]:
# Remove the indexes
drop_index = 'DROP INDEX index_medallion; DROP INDEX index_passenger;'
cursor_postgres.execute(drop_index)

In [ ]:
# cluster = 'CLUSTER TripData using index_medallion;'
# cursor_postgres.execute(cluster)
# print(cursor_postgres.fetchall())

### Query 0

In [9]:
# ------------------------- Query 0 -------------------------
import time


print("                                   PostgreSQL\n-------------------------------------------------------------------------------")

query = 'SELECT * FROM TripData WHERE medallion = \'0B57B9633A2FECD3D3B1944AFC7471CF\';'

explain = 'EXPLAIN ANALYZE ' + query
cursor_postgres.execute(explain)
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
execution_plan = cursor_postgres.fetchall()
for line in execution_plan:
    print(line[0])
    
print(execution_plan)
print("-------------------------------------------------------------------------------")


print("                                    QUERY 0")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_postgres.execute(query)
    # print(cursor_postgres.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

print("-------------------------------------------------------------------------------")

# print(cursor_postgres.fetchall())

                                   PostgreSQL
-------------------------------------------------------------------------------
                                   QUERY PLAN
-------------------------------------------------------------------------------
Gather  (cost=1000.00..399428.24 rows=1147 width=140) (actual time=11.978..1483.911 rows=1512 loops=1)
  Workers Planned: 2
  Workers Launched: 2
  ->  Parallel Seq Scan on tripdata  (cost=0.00..398313.54 rows=478 width=140) (actual time=7.455..1463.798 rows=504 loops=3)
        Filter: ((medallion)::text = '0B57B9633A2FECD3D3B1944AFC7471CF'::text)
        Rows Removed by Filter: 4925034
Planning Time: 0.451 ms
JIT:
  Functions: 6
  Options: Inlining false, Optimization false, Expressions true, Deforming true
  Timing: Generation 0.610 ms, Inlining 0.000 ms, Optimization 0.535 ms, Emission 8.549 ms, Total 9.693 ms
Execution Time: 1484.434 ms
[('Gather  (cost=1000.00..399428.24 rows=1147 width=140) (actual time=11.978..1483.911 rows=1512 l

In [54]:
print("                                                      DuckDB\n--------------------------------------------------------------------------------")

duckDBquery = 'SELECT * FROM TripData WHERE medallion = \'00005007A9F30E289E760362F69E4EAD\';'

explain = 'EXPLAIN ANALYZE ' + duckDBquery
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
cursor_duckdb.execute(explain)
execution_plan = cursor_duckdb.fetchall()
print(execution_plan)
print("-------------------------------------------------------------------------------")


print("                                    QUERY 0")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_duckdb.execute(duckDBquery)
    # print(cursor_duckdb.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

# print("-------------------------------------------------------------------------------")
# print(cursor_duckdb.fetchall())

                                                      DuckDB
--------------------------------------------------------------------------------------------------------------------
                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
[('analyzed_plan', "┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││    Query Profiling Information    ││\n│└───────────────────────────────────┘│\n└─────────────────────────────────────┘\nEXPLAIN ANALYZE SELECT * FROM TripData WHERE medallion = '00005007A9F30E289E760362F69E4EAD';\n┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││        Total Time: 0.0157s        ││\n│└───────────────────────────────────┘│\n└─────────────────────────────────────┘\n┌───────────────────────────┐\n│      EXPLAIN_ANALYZE      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│             0         

### Query 1

In [10]:
# ------------------------- Query 1 -------------------------
print("                                   PostgreSQL\n-------------------------------------------------------------------------------")

query = 'SELECT * FROM TripData WHERE passenger_count >= 2;'

explain = 'EXPLAIN ANALYZE ' + query
cursor_postgres.execute(explain)
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
execution_plan = cursor_postgres.fetchall()
for line in execution_plan:
    print(line[0])
print("-------------------------------------------------------------------------------")

print("                                    QUERY 1")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_postgres.execute(query)
    # print(cursor_postgres.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                   PostgreSQL
-------------------------------------------------------------------------------
                                   QUERY PLAN
-------------------------------------------------------------------------------
Seq Scan on tripdata  (cost=0.00..506059.69 rows=4274875 width=140) (actual time=82.609..2785.382 rows=4304748 loops=1)
  Filter: (passenger_count >= 2)
  Rows Removed by Filter: 10471867
Planning Time: 0.100 ms
JIT:
  Functions: 2
  Options: Inlining true, Optimization true, Expressions true, Deforming true
  Timing: Generation 0.216 ms, Inlining 45.297 ms, Optimization 20.676 ms, Emission 16.620 ms, Total 82.809 ms
Execution Time: 2900.227 ms
-------------------------------------------------------------------------------
                                    QUERY 1
Average Time Elapsed / sql execution (in sec):
5.062296803599975


In [55]:
# Query 1

print("                                                      DuckDB\n--------------------------------------------------------------------------------")

query = 'SELECT * FROM TripData WHERE passenger_count >= 2;'

explain = 'EXPLAIN ANALYZE ' + duckDBquery
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
cursor_duckdb.execute(explain)
execution_plan = cursor_duckdb.fetchall()
print(execution_plan)
print("-------------------------------------------------------------------------------")

print("                                    QUERY 1")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_duckdb.execute(query)
    # print(cursor_duckdb.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                                      DuckDB
--------------------------------------------------------------------------------------------------------------------
                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
[('analyzed_plan', "┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││    Query Profiling Information    ││\n│└───────────────────────────────────┘│\n└─────────────────────────────────────┘\nEXPLAIN ANALYZE SELECT * FROM TripData WHERE medallion = '00005007A9F30E289E760362F69E4EAD';\n┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││        Total Time: 0.0163s        ││\n│└───────────────────────────────────┘│\n└─────────────────────────────────────┘\n┌───────────────────────────┐\n│      EXPLAIN_ANALYZE      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│             0         

### Query 2

In [14]:
# ------------------------- Query 2 -------------------------
import time

print("                                   PostgreSQL\n-------------------------------------------------------------------------------")

query = 'SELECT * FROM TripData WHERE passenger_count = 1;'

explain = 'EXPLAIN ANALYZE ' + query
cursor_postgres.execute(explain)
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
execution_plan = cursor_postgres.fetchall()
for line in execution_plan:
    print(line[0])
print("-------------------------------------------------------------------------------")

print("                                    QUERY 2")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_postgres.execute(query)
    # print(cursor_postgres.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                   PostgreSQL
-------------------------------------------------------------------------------
                                   QUERY PLAN
-------------------------------------------------------------------------------
Seq Scan on tripdata  (cost=0.00..506059.69 rows=10501740 width=140) (actual time=95.853..2392.197 rows=10471701 loops=1)
  Filter: (passenger_count = 1)
  Rows Removed by Filter: 4304914
Planning Time: 0.205 ms
JIT:
  Functions: 2
  Options: Inlining true, Optimization true, Expressions true, Deforming true
  Timing: Generation 0.231 ms, Inlining 24.688 ms, Optimization 47.738 ms, Emission 23.404 ms, Total 96.061 ms
Execution Time: 2666.169 ms
-------------------------------------------------------------------------------
                                    QUERY 2
Average Time Elapsed / sql execution (in sec):
19.501642151800024


In [56]:
# ------------------------- Query 2 -------------------------

print("                                                     DuckDB\n--------------------------------------------------------------------------------")

query = 'SELECT * FROM TripData WHERE passenger_count = 1;'

explain = 'EXPLAIN ANALYZE ' + duckDBquery
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
cursor_duckdb.execute(explain)
execution_plan = cursor_duckdb.fetchall()
print(execution_plan)
print("-------------------------------------------------------------------------------")

print("                                    QUERY 2")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_duckdb.execute(query)
    # print(cursor_duckdb.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                                     DuckDB
--------------------------------------------------------------------------------------------------------------------
                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
[('analyzed_plan', "┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││    Query Profiling Information    ││\n│└───────────────────────────────────┘│\n└─────────────────────────────────────┘\nEXPLAIN ANALYZE SELECT * FROM TripData WHERE medallion = '00005007A9F30E289E760362F69E4EAD';\n┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││        Total Time: 0.0153s        ││\n│└───────────────────────────────────┘│\n└─────────────────────────────────────┘\n┌───────────────────────────┐\n│      EXPLAIN_ANALYZE      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│             0          

### Aggreagte 0

In [5]:
# ------------------------- Aggregate 0 -------------------------
import time

print("                                   PostgreSQL\n-------------------------------------------------------------------------------")

aggregate = 'SELECT SUM(trip_distance) FROM TripData;'

explain = 'EXPLAIN ANALYZE ' + aggregate
cursor_postgres.execute(explain)
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
execution_plan = cursor_postgres.fetchall()
for line in execution_plan:
    print(line[0])
print("-------------------------------------------------------------------------------")

print("                                   AGGREGATE 0")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_postgres.execute(aggregate)
    # print(cursor_postgres.fetchall())


end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                   PostgreSQL
-------------------------------------------------------------------------------
                                   QUERY PLAN
-------------------------------------------------------------------------------
Finalize Aggregate  (cost=399313.75..399313.76 rows=1 width=8) (actual time=729.125..733.073 rows=1 loops=1)
  ->  Gather  (cost=399313.54..399313.75 rows=2 width=8) (actual time=729.000..733.054 rows=3 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Partial Aggregate  (cost=398313.54..398313.55 rows=1 width=8) (actual time=716.973..716.974 rows=1 loops=3)
              ->  Parallel Seq Scan on tripdata  (cost=0.00..382921.23 rows=6156923 width=8) (actual time=0.027..400.020 rows=4925538 loops=3)
Planning Time: 0.082 ms
JIT:
  Functions: 11
  Options: Inlining false, Optimization false, Expressions true, Deforming true
  Timing: Generation 0.729 ms, Inlining 0.000 ms, Optimization 0.621 ms, Emission 11.152 ms, 

In [ ]:
# ------------------------- Aggregate 0 -------------------------
print("                                                     DuckDB\n--------------------------------------------------------------------------------")

aggregate = 'SELECT SUM(trip_distance) FROM TripData;'

explain = 'EXPLAIN ANALYZE ' + aggregate
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
cursor_duckdb.execute(explain)
execution_plan = cursor_duckdb.fetchall()
print(execution_plan)
print("-------------------------------------------------------------------------------")

print("                                                     AGGREGATE 0")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_duckdb.execute(aggregate)
    # print(cursor_duckdb.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

### Aggregate 1

In [17]:
# ------------------------- Aggregate 1 -------------------------
import time

print("                                   PostgreSQL\n-------------------------------------------------------------------------------")

aggregate = 'SELECT medallion, AVG(trip_distance) FROM TripData WHERE passenger_count = 1 GROUP BY medallion;'

explain = 'EXPLAIN ANALYZE ' + aggregate
cursor_postgres.execute(explain)
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
execution_plan = cursor_postgres.fetchall()
for line in execution_plan:
    print(line[0])
print("-------------------------------------------------------------------------------")

print("                                                     AGGREGATE 1")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_postgres.execute(aggregate)
    # print(cursor_postgres.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                   PostgreSQL
-------------------------------------------------------------------------------
                                   QUERY PLAN
-------------------------------------------------------------------------------
Finalize GroupAggregate  (cost=422193.90..425470.31 rows=12806 width=41) (actual time=1970.244..1987.353 rows=13348 loops=1)
  Group Key: medallion
  ->  Gather Merge  (cost=422193.90..425182.18 rows=25612 width=65) (actual time=1970.170..1981.676 rows=39579 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Sort  (cost=421193.88..421225.90 rows=12806 width=65) (actual time=1951.308..1952.975 rows=13193 loops=3)
              Sort Key: medallion
              Sort Method: quicksort  Memory: 1724kB
              Worker 0:  Sort Method: quicksort  Memory: 1724kB
              Worker 1:  Sort Method: quicksort  Memory: 1726kB
              ->  Partial HashAggregate  (cost=420192.16..420320.22 rows=12806 width=65) (ac

In [59]:
# ------------------------- Aggregate 1 -------------------------
print("                                                     DuckDB\n--------------------------------------------------------------------------------")

aggregate = 'SELECT medallion, AVG(trip_distance) FROM TripData WHERE passenger_count = 1 GROUP BY medallion;'

explain = 'EXPLAIN ANALYZE ' + aggregate
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
cursor_duckdb.execute(explain)
execution_plan = cursor_duckdb.fetchall()
print(execution_plan)
print("-------------------------------------------------------------------------------")

print("                                                     AGGREGATE 1")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_duckdb.execute(aggregate)
    # print(cursor_duckdb.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                                     DuckDB
--------------------------------------------------------------------------------------------------------------------
                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
[('analyzed_plan', '┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││    Query Profiling Information    ││\n│└───────────────────────────────────┘│\n└─────────────────────────────────────┘\nEXPLAIN ANALYZE SELECT medallion, AVG(trip_distance) FROM TripData WHERE passenger_count = 1 GROUP BY medallion;\n┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││         Total Time: 0.159s        ││\n│└───────────────────────────────────┘│\n└─────────────────────────────────────┘\n┌───────────────────────────┐\n│      EXPLAIN_ANALYZE      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│    

### Aggregate 2

In [18]:
# ------------------------- Aggregate 2 -------------------------
import time

print("                                   PostgreSQL\n-------------------------------------------------------------------------------")

aggregate = 'SELECT AVG(passenger_count) FROM TripData GROUP BY pickup_datetime;'

explain = 'EXPLAIN ANALYZE ' + aggregate
cursor_postgres.execute(explain)
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
execution_plan = cursor_postgres.fetchall()
for line in execution_plan:
    print(line[0])
print("-------------------------------------------------------------------------------")

print("                                   AGGREGATE 2")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_postgres.execute(aggregate)
    # print(cursor_postgres.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                   PostgreSQL
-------------------------------------------------------------------------------
                                   QUERY PLAN
-------------------------------------------------------------------------------
Finalize GroupAggregate  (cost=856814.49..894876.22 rows=148766 width=40) (actual time=23679.484..26141.667 rows=2303465 loops=1)
  Group Key: pickup_datetime
  ->  Gather Merge  (cost=856814.49..891528.98 rows=297532 width=40) (actual time=23679.318..25234.041 rows=4525799 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Sort  (cost=855814.47..856186.38 rows=148766 width=40) (actual time=22633.304..23094.821 rows=1508600 loops=3)
              Sort Key: pickup_datetime
              Sort Method: external merge  Disk: 80408kB
              Worker 0:  Sort Method: external merge  Disk: 83048kB
              Worker 1:  Sort Method: external merge  Disk: 80224kB
              ->  Partial HashAggregate  (cost=777349

In [61]:
# ------------------------- Aggregate 2 -------------------------
import time

print("                                                     DuckDB\n--------------------------------------------------------------------------------")

aggregate = 'SELECT AVG(passenger_count) FROM TripData GROUP BY pickup_datetime;'

explain = 'EXPLAIN ANALYZE ' + aggregate
cursor_duckdb.execute(explain)
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
print(cursor_duckdb.fetchall())
print("-------------------------------------------------------------------------------")

print("                                                     AGGREGATE 2")
number_of_iteration = 5;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_duckdb.execute(aggregate)
    # print(cursor_duckdb.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                                     DuckDB
--------------------------------------------------------------------------------------------------------------------
                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
[('analyzed_plan', '┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││    Query Profiling Information    ││\n│└───────────────────────────────────┘│\n└─────────────────────────────────────┘\nEXPLAIN ANALYZE SELECT AVG(passenger_count) FROM TripData GROUP BY pickup_datetime;\n┌─────────────────────────────────────┐\n│┌───────────────────────────────────┐│\n││         Total Time: 0.210s        ││\n│└───────────────────────────────────┘│\n└─────────────────────────────────────┘\n┌───────────────────────────┐\n│      EXPLAIN_ANALYZE      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│             0             │\n│  

### Aggregate 3

In [19]:
# ------------------------- Aggregate 3 -------------------------
import time

print("                                   PostgreSQL\n-------------------------------------------------------------------------------")

# Taxis' stats
query = 'SELECT medallion, AVG(passenger_count) as avg_pass_count, AVG(trip_distance) as avg_trip_dist FROM TripData GROUP BY medallion'
global_avg = 'SELECT AVG(trip_distance) FROM TripData'

aggregate = 'SELECT avg_pass_count FROM (' + query + ') as taxisStats WHERE avg_trip_dist > (' + global_avg + ');'

explain = 'EXPLAIN ANALYZE ' + aggregate
cursor_postgres.execute(explain)
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
execution_plan = cursor_postgres.fetchall()
for line in execution_plan:
    print(line[0])
print("-------------------------------------------------------------------------------")


print("                                  AGGREGATE 3")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):

    # cursor_postgres.execute(query)
    # print('(medallion, avg_passenger_count, avg_trip_dist):')
    # print(cursor_postgres.fetchall())
    # print("-------------")

    # cursor_postgres.execute(global_avg)
    # print('global avg trip distance:')
    # print(cursor_postgres.fetchall())
    # print("-------------")

    cursor_postgres.execute(aggregate)
    # print('A3:')
    # print(cursor_postgres.fetchall())
    # print("-------------")


end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                   PostgreSQL
-------------------------------------------------------------------------------
                                   QUERY PLAN
-------------------------------------------------------------------------------
Subquery Scan on taxisstats  (cost=830413.66..833860.82 rows=4269 width=32) (actual time=3738.176..3760.707 rows=5588 loops=1)
  InitPlan 1 (returns $1)
    ->  Finalize Aggregate  (cost=399313.75..399313.76 rows=1 width=8) (actual time=1367.873..1367.932 rows=1 loops=1)
          ->  Gather  (cost=399313.54..399313.75 rows=2 width=32) (actual time=1367.751..1367.913 rows=3 loops=1)
                Workers Planned: 2
                Workers Launched: 2
                ->  Partial Aggregate  (cost=398313.54..398313.55 rows=1 width=32) (actual time=1354.580..1354.581 rows=1 loops=3)
                      ->  Parallel Seq Scan on tripdata tripdata_1  (cost=0.00..382921.23 rows=6156923 width=8) (actual time=0.010..1014.309 rows=4925538 loops=

In [ ]:
# ------------------------- Aggregate 3 -------------------------
import time

print("                                                     DuckDB\n--------------------------------------------------------------------------------")

# Taxis' stats
query = 'SELECT medallion, AVG(passenger_count) as avg_pass_count, AVG(trip_distance) as avg_trip_dist FROM TripData GROUP BY medallion'
global_avg = 'SELECT AVG(trip_distance) FROM TripData'

aggregate = 'SELECT avg_pass_count FROM (' + query + ') as taxisStats WHERE avg_trip_dist > (' + global_avg + ');'

explain = 'EXPLAIN ANALYZE ' + aggregate
cursor_duckdb.execute(explain)
print("                                   QUERY PLAN\n-------------------------------------------------------------------------------")
print(cursor_duckdb.fetchall())
print("-------------------------------------------------------------------------------")


print("                                                     AGGREGATE 3")
number_of_iteration = 10;
start_time = time.perf_counter()

for x in range(number_of_iteration):
    cursor_duckdb.execute(aggregate)

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

                                                     DuckDB
--------------------------------------------------------------------------------------------------------------------
                                                     QUERY PLAN
--------------------------------------------------------------------------------------------------------------------
[]
--------------------------------------------------------------------------------------------------------------------
                                                     AGGREGATE 3


## Comparison

|                              |Query 0    |Query 1    |Query 2    |Aggregate 1|Aggregate 2|Aggregate 3|
|------------------------------|-----------|-----------|-----------|-----------|-----------|-----------|
|postgresql no opti            | 2.6890s   | 15.3270s  | 20.8068s  | 5.4380s   | 28.3206s ~ 19.3993s  | 6.0251s   |
|postgresql index on medallion |           |           |           | 6.2733s   | 24.6855s  | 8.1786s   |
|duckDB no opti                | 0.0003s   | 0.0004s   | 0.0003s   | 0.0885s   | 0.1041s   | 0.0017s   |

## Physical Plan (in the first archive)

### PostgreSQL

- Query 0
    ```
    Bitmap Heap Scan on tripdata  (cost=17.50..4433.00 rows=1154 width=140)
  Recheck Cond: ((medallion)::text = '0BD7C8F5BA12B88E0B67BED28BEA73D8'::text)
  ->  Bitmap Index Scan on index  (cost=0.00..17.21 rows=1154 width=0)
        Index Cond: ((medallion)::text = '0BD7C8F5BA12B88E0B67BED28BEA73D8'::text)
    ```
- Query 1
    ```
    Seq Scan on tripdata  (cost=0.00..506058.69 rows=4308861 width=140)
  Filter: (passenger_count >= 2)
JIT:
  Functions: 2
  Options: Inlining true, Optimization true, Expressions true, Deforming true
    ```
- Query 2
    ```
    Seq Scan on tripdata  (cost=0.00..506058.69 rows=10467754 width=140)
  Filter: (passenger_count = 1)
JIT:
  Functions: 2
  Options: Inlining true, Optimization true, Expressions true, Deforming true
    ```
- Aggregate 1
    ```
    Finalize GroupAggregate  (cost=422117.32..425379.40 rows=12750 width=41)
  Group Key: medallion
  ->  Gather Merge  (cost=422117.32..425092.52 rows=25500 width=65)
        Workers Planned: 2
        ->  Sort  (cost=421117.29..421149.17 rows=12750 width=65)
              Sort Key: medallion
              ->  Partial HashAggregate  (cost=420120.36..420247.86 rows=12750 width=65)
                    Group Key: medallion
                    ->  Parallel Seq Scan on tripdata  (cost=0.00..398312.54 rows=4361564 width=41)
                          Filter: (passenger_count = 1)
JIT:
  Functions: 9
  Options: Inlining false, Optimization false, Expressions true, Deforming true
    ```
- Aggregate 2
    ```
    Finalize GroupAggregate  (cost=856301.62..893361.70 rows=144851 width=40)
  Group Key: pickup_datetime
  ->  Gather Merge  (cost=856301.62..890102.55 rows=289702 width=40)
        Workers Planned: 2
        ->  Sort  (cost=855301.60..855663.73 rows=144851 width=40)
              Sort Key: pickup_datetime
              ->  Partial HashAggregate  (cost=777348.11..838922.82 rows=144851 width=40)
                    Group Key: pickup_datetime
                    Planned Partitions: 4
                    ->  Parallel Seq Scan on tripdata  (cost=0.00..382920.23 rows=6156923 width=12)
JIT:
  Functions: 7
  Options: Inlining true, Optimization true, Expressions true, Deforming true
    ```
- Aggregate 3
    ```
    Subquery Scan on taxisstats  (cost=830406.88..833838.96 rows=4250 width=32)
  InitPlan 1 (returns $1)
    ->  Finalize Aggregate  (cost=399312.75..399312.76 rows=1 width=8)
          ->  Gather  (cost=399312.54..399312.75 rows=2 width=32)
                Workers Planned: 2
                ->  Partial Aggregate  (cost=398312.54..398312.55 rows=1 width=32)
                      ->  Parallel Seq Scan on tripdata tripdata_1  (cost=0.00..382920.23 rows=6156923 width=8)
  ->  Finalize GroupAggregate  (cost=431094.11..434483.69 rows=4250 width=73)
        Group Key: tripdata.medallion
        Filter: (avg(tripdata.trip_distance) > $1)
        ->  Gather Merge  (cost=431094.11..434069.32 rows=25500 width=97)
              Workers Planned: 2
              ->  Sort  (cost=430094.09..430125.96 rows=12750 width=97)
                    Sort Key: tripdata.medallion
                    ->  Partial HashAggregate  (cost=429097.15..429224.65 rows=12750 width=97)
                          Group Key: tripdata.medallion
                          ->  Parallel Seq Scan on tripdata  (cost=0.00..382920.23 rows=6156923 width=45)
JIT:
  Functions: 14
  Options: Inlining true, Optimization true, Expressions true, Deforming true
    ```

### DuckDB

- Query 0
    ```
    [('physical_plan', '┌───────────────────────────┐\n│         SEQ_SCAN          │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│       TripDataSample      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│         medallion         │\n│        hack_licence       │\n│         vendor_id         │\n│         rate_code         │\n│     store_and_fwd_flag    │\n│      pickup_datetime      │\n│      dropoff_datetime     │\n│      passenger_count      │\n│     trip_time_in_secs     │\n│       trip_distance       │\n│      pickup_longitude     │\n│      pickup_latitude      │\n│     dropoff_longitude     │\n│      dropoff_latitude     │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│     Filters: medallion    │\n│=0BD7C8F5BA12B88E0B67B...  │\n│  AND medallion IS NOT NULL│\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│           EC: 2           │\n└───────────────────────────┘                             \n')]
    ```
- Query 1
    ```
    [('physical_plan', '┌───────────────────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│         medallion         │\n│        hack_licence       │\n│         vendor_id         │\n│         rate_code         │\n│     store_and_fwd_flag    │\n│      pickup_datetime      │\n│      dropoff_datetime     │\n│      passenger_count      │\n│     trip_time_in_secs     │\n│       trip_distance       │\n│      pickup_longitude     │\n│      pickup_latitude      │\n│     dropoff_longitude     │\n│      dropoff_latitude     │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│         SEQ_SCAN          │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│       TripDataSample      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│      passenger_count      │\n│         medallion         │\n│        hack_licence       │\n│         vendor_id         │\n│         rate_code         │\n│     store_and_fwd_flag    │\n│      pickup_datetime      │\n│      dropoff_datetime     │\n│     trip_time_in_secs     │\n│       trip_distance       │\n│      pickup_longitude     │\n│      pickup_latitude      │\n│     dropoff_longitude     │\n│      dropoff_latitude     │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│Filters: passenger_count>=2│\n│ AND passenger_count IS NOT│\n│            NULL           │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│           EC: 1           │\n└───────────────────────────┘                             \n')]
    ```
- Query 2
    ```
    [('physical_plan', '┌───────────────────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│         medallion         │\n│        hack_licence       │\n│         vendor_id         │\n│         rate_code         │\n│     store_and_fwd_flag    │\n│      pickup_datetime      │\n│      dropoff_datetime     │\n│      passenger_count      │\n│     trip_time_in_secs     │\n│       trip_distance       │\n│      pickup_longitude     │\n│      pickup_latitude      │\n│     dropoff_longitude     │\n│      dropoff_latitude     │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│         SEQ_SCAN          │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│       TripDataSample      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│      passenger_count      │\n│         medallion         │\n│        hack_licence       │\n│         vendor_id         │\n│         rate_code         │\n│     store_and_fwd_flag    │\n│      pickup_datetime      │\n│      dropoff_datetime     │\n│     trip_time_in_secs     │\n│       trip_distance       │\n│      pickup_longitude     │\n│      pickup_latitude      │\n│     dropoff_longitude     │\n│      dropoff_latitude     │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│ Filters: passenger_count=1│\n│ AND passenger_count IS NOT│\n│            NULL           │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│           EC: 3           │\n└───────────────────────────┘                             \n')]
    ```
- Aggregate 1
    ```
    [('physical_plan', '┌───────────────────────────┐\n│       HASH_GROUP_BY       │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│             #0            │\n│          avg(#1)          │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│         medallion         │\n│   CAST(trip_distance AS   │\n│           DOUBLE)         │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│         SEQ_SCAN          │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│       TripDataSample      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│         medallion         │\n│       trip_distance       │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│ Filters: passenger_count=1│\n│ AND passenger_count IS NOT│\n│            NULL           │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│           EC: 3           │\n└───────────────────────────┘                             \n')]
    ```
- Aggregate 2
    ```
    [('physical_plan', '┌───────────────────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│    avg(passenger_count)   │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│       HASH_GROUP_BY       │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│             #0            │\n│          avg(#1)          │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│      pickup_datetime      │\n│      passenger_count      │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│         SEQ_SCAN          │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│       TripDataSample      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│      pickup_datetime      │\n│      passenger_count      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│           EC: 9           │\n└───────────────────────────┘                             \n')]
    ```
- Aggregate 3
    ```
    [('physical_plan', '┌───────────────────────────┐                             \n│         PROJECTION        │                             \n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             \n│       avg_pass_count      │                             \n└─────────────┬─────────────┘                                                          \n┌─────────────┴─────────────┐                             \n│      NESTED_LOOP_JOIN     │                             \n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             \n│           INNER           │                             \n│  avg_trip_dist > SUBQUERY ├──────────────┐              \n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │              │              \n│           EC: 40          │              │              \n└─────────────┬─────────────┘              │                                           \n┌─────────────┴─────────────┐┌─────────────┴─────────────┐\n│         PROJECTION        ││    UNGROUPED_AGGREGATE    │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│       avg_pass_count      ││         first(#0)         │\n│       avg_trip_dist       ││                           │\n└─────────────┬─────────────┘└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐┌─────────────┴─────────────┐\n│       HASH_GROUP_BY       ││         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│             #0            ││             #0            │\n│          avg(#1)          ││                           │\n│          avg(#2)          ││                           │\n└─────────────┬─────────────┘└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐┌─────────────┴─────────────┐\n│         PROJECTION        ││      STREAMING_LIMIT      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││                           │\n│         medallion         ││                           │\n│      passenger_count      ││                           │\n│   CAST(trip_distance AS   ││                           │\n│           DOUBLE)         ││                           │\n└─────────────┬─────────────┘└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐┌─────────────┴─────────────┐\n│         SEQ_SCAN          ││    UNGROUPED_AGGREGATE    │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│       TripDataSample      ││          avg(#0)          │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││                           │\n│         medallion         ││                           │\n│      passenger_count      ││                           │\n│       trip_distance       ││                           │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   ││                           │\n│           EC: 18          ││                           │\n└───────────────────────────┘└─────────────┬─────────────┘                             \n                             ┌─────────────┴─────────────┐\n                             │         PROJECTION        │\n                             │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n                             │   CAST(trip_distance AS   │\n                             │           DOUBLE)         │\n                             └─────────────┬─────────────┘                             \n                             ┌─────────────┴─────────────┐\n                             │         SEQ_SCAN          │\n                             │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n                             │       TripDataSample      │\n                             │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n                             │       trip_distance       │\n                             │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n                             │           EC: 18          │\n                             └───────────────────────────┘                             \n')]
    ```

## Compare DuckDB and PostgreSQL

- OLAP vs OLTP

In [ ]:
# Closing the connection 
conn.close()